# Práctica 3

Considere un problema de búsqueda con estados $\{A, B, C, D, E, F, G, H, I, J\}$ . Donde $A$ es el
estado inicial y J el único estado objetivo. Las acciones que se pueden ejecutar en cada estado se
especifican por una pareja $(s, c)$ donde $s$ es el estado resultante de ejecutar la acción y $c$ es el costo
de la acción:

\begin{align*}
A(h=8)   &: (B, 4),\ (E, 2),\ (H, 3) \\
B(h=3)   &: (C, 4),\ (D, 2),\ (E, 1),\ (G, 4),\ (I, 5) \\
C(h=2)   &: (E, 1),\ (F, 3),\ (G, 3),\ (I, 2) \\
D(h=4)   &: (A, 3),\ (C, 2),\ (E, 5),\ (F, 4),\ (H, 5) \\
E(h=10)  &: (D, 5) \\
F(h=10)  &: (B, 4),\ (H, 5) \\
G(h=13)  &: (H, 1) \\
H(h=12)  &: (F, 4),\ (G, 2) \\
I(h=0)   &: (C, 5),\ (G, 1),\ (H, 4),\ (J, 1) \\
J(h=0)   &: \\
\end{align*}

El valor en paréntesis al frente de cada estado corresponde al valor de la heurística h.

In [9]:
def get_structure(s):
    new_s = [a for a,_ in s]
    return new_s

class Graph:
    def __init__(self,costed_edges,heuristics):
        self.costed_edges = costed_edges
        self.heuristics = heuristics

        self.adj_list = {i:set() for i in self.heuristics}

        for a,b in costed_edges.keys():
            self.adj_list[a].add(b)

    def defineSortCriteria(self,option = None):
        flag = False
        if option == 'ascending label':
            for node, adj in self.adj_list.items():
                l = list(adj)
                l.sort()
                self.adj_list[node] = l
            flag = True
        elif option == 'descending label':
            for node, adj in self.adj_list.items():
                l = list(adj)
                l.sort(reverse = True)
                self.adj_list[node] = l
            flag = True
        elif option is None:
            for node, adj in self.adj_list.items():
                l = list(adj)
                self.adj_list[node] = l
            flag = True
        return flag

    def contains(self,node):
        return node in self.adj_list.keys()
    
    def h(self,node):
        if node not in self.heuristics.keys(): return None
        return self.heuristics[node]

    def cost(self,a,b):
        if (a,b) not in self.costed_edges.keys(): return None
        return  self.costed_edges[(a,b)]

    def adj(self,node):
        if node not in self.adj_list.keys(): return None
        return self.adj_list[node]
    
    def print_structure(self):
        nodes = sorted(self.adj_list.keys())
        max_name_len = max(len(str(n)) for n in nodes)
        max_h_len = max(len(str(self.h(n))) for n in nodes)
        for node in nodes:
            neighbours = sorted(self.adj(node))
            strings = [f'({n},{self.cost(node,n)})' for n in neighbours]
            s = ', '.join(strings)
            node_str = str(node).ljust(max_name_len)
            h_str = str(self.h(node)).rjust(max_h_len)
            print(f'{node_str}(h = {h_str})  :  {s}')


In [10]:
h = {
    'A': 8, 'B': 3, 'C': 2, 'D': 4, 'E': 10,
    'F': 10, 'G': 13, 'H': 12, 'I': 0, 'J': 0
}

c = {
    ('A', 'B'): 4, ('A', 'E'): 2, ('A', 'H'): 3,
    ('B', 'C'): 4, ('B', 'D'): 2, ('B', 'E'): 1, ('B', 'G'): 4, ('B', 'I'): 5,
    ('C', 'E'): 1, ('C', 'F'): 3, ('C', 'G'): 3, ('C', 'I'): 2,
    ('D', 'A'): 3, ('D', 'C'): 2, ('D', 'E'): 5, ('D', 'F'): 4, ('D', 'H'): 5,
    ('E', 'D'): 5,
    ('F', 'B'): 4, ('F', 'H'): 5,
    ('G', 'H'): 1,
    ('H', 'F'): 4, ('H', 'G'): 2,
    ('I', 'C'): 5, ('I', 'G'): 1, ('I', 'H'): 4, ('I', 'J'): 1
}

G = Graph(c,h)
G.print_structure()

A(h =  8)  :  (B,4), (E,2), (H,3)
B(h =  3)  :  (C,4), (D,2), (E,1), (G,4), (I,5)
C(h =  2)  :  (E,1), (F,3), (G,3), (I,2)
D(h =  4)  :  (A,3), (C,2), (E,5), (F,4), (H,5)
E(h = 10)  :  (D,5)
F(h = 10)  :  (B,4), (H,5)
G(h = 13)  :  (H,1)
H(h = 12)  :  (F,4), (G,2)
I(h =  0)  :  (C,5), (G,1), (H,4), (J,1)
J(h =  0)  :  


## 1. Búsqueda en profundidad (DFS)

Ejecute búsqueda en profundidad, teniendo en cuenta que las estados entran a la pila en orden inverso de su etiqueta, de manera que al extraerlos de la pila saldría el estado con menor etiqueta. Igualmente, tenga en cuenta que el algoritmo de búsqueda previene ciclos llevando una lista de nodos alcanzados (reached).

In [11]:
def DFSearch(G,start,end, criteria = None, verbose = False):
    assert G.contains(start) and G.contains(end) and G.defineSortCriteria(option = criteria)
    stack = [(start , [start])]
    out = []
    reached = set()
    while len(stack) > 0:
        if verbose: 
            fridge = get_structure(stack)
            print(f'Out: {fridge[-1]}  -  Fridge: {fridge}')
        cur,path = stack.pop()
        out.append(cur)
        if cur == end:
            if verbose: print(f'\nOut order: {" ".join(out)}') 
            return path
        if cur not in reached:
            reached.add(cur)
            for node in G.adj(cur):
                stack.append((node , path + [node]))
    if verbose: print(f'\nOut order: {" ".join(out)}')
    return None

In [12]:
path = DFSearch(G,'A','J',verbose = True)
if path: print('\nFinal path found: '+' -> '.join(path))

Out: A  -  Fridge: ['A']
Out: H  -  Fridge: ['E', 'B', 'H']
Out: G  -  Fridge: ['E', 'B', 'F', 'G']
Out: H  -  Fridge: ['E', 'B', 'F', 'H']
Out: F  -  Fridge: ['E', 'B', 'F']
Out: B  -  Fridge: ['E', 'B', 'H', 'B']
Out: E  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'E']
Out: D  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'D']
Out: E  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'C', 'A', 'F', 'H', 'E']
Out: H  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'C', 'A', 'F', 'H']
Out: F  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'C', 'A', 'F']
Out: A  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'C', 'A']
Out: C  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'C']
Out: G  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'I', 'F', 'E', 'G']
Out: E  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'I', 'F', 'E']
Out: F  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'I', 'F']
Out: I  -  Fridge: ['E', 'B', 'H', 'C', 'I', 'D', 'G', 'I']
Out: C  -  Fridge: ['E', 'B', 'H', 'C', 'I', 

## 2. Búsqueda en amplitud (BFS)

Ejecute búsqueda en amplitud, teniendo en cuenta que las estados entran a la cola en el orden
de su etiqueta.

In [13]:
def BFSearch(G,start,end, criteria = None, verbose = False):
    assert G.contains(start) and G.contains(end) and G.defineSortCriteria(option = criteria)
    queue = [(start , [start])]
    out = []
    reached = set()
    while len(queue) > 0:
        if verbose: 
            fridge = get_structure(queue)
            print(f'Out: {fridge[0]}  -  Fridge: {fridge}')
        cur,path = queue.pop(0)
        out.append(cur)
        if cur == end:
            if verbose: print(f'\nOut order: {" ".join(out)}')
            return path
        if cur not in reached:
            reached.add(cur)
            for node in G.adj(cur):
                queue.append((node , path + [node]))
    return None

In [14]:
path = BFSearch(G,'A','J',verbose = True)
if path: print('\nFinal path found: '+' -> '.join(path))

Out: A  -  Fridge: ['A']
Out: E  -  Fridge: ['E', 'B', 'H']
Out: B  -  Fridge: ['B', 'H', 'D']
Out: H  -  Fridge: ['H', 'D', 'C', 'I', 'D', 'G', 'E']
Out: D  -  Fridge: ['D', 'C', 'I', 'D', 'G', 'E', 'F', 'G']
Out: C  -  Fridge: ['C', 'I', 'D', 'G', 'E', 'F', 'G', 'C', 'A', 'F', 'H', 'E']
Out: I  -  Fridge: ['I', 'D', 'G', 'E', 'F', 'G', 'C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G']
Out: D  -  Fridge: ['D', 'G', 'E', 'F', 'G', 'C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G', 'J', 'G', 'H', 'C']
Out: G  -  Fridge: ['G', 'E', 'F', 'G', 'C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G', 'J', 'G', 'H', 'C']
Out: E  -  Fridge: ['E', 'F', 'G', 'C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G', 'J', 'G', 'H', 'C', 'H']
Out: F  -  Fridge: ['F', 'G', 'C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G', 'J', 'G', 'H', 'C', 'H']
Out: G  -  Fridge: ['G', 'C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G', 'J', 'G', 'H', 'C', 'H', 'H', 'B']
Out: C  -  Fridge: ['C', 'A', 'F', 'H', 'E', 'I', 'F', 'E', 'G', 'J', 'G', 'H', 'C', 'H', 

## 3. Búsqueda de costo uniforme (UCS)

Ahora ejecute búsqueda de costo uniforme. Tenga en cuenta que si hay empates en la cola de prioridad se extrae el estado con menor etiqueta.

In [15]:
import heapq

def UCSearch(G,start,end, criteria = None, verbose = False):
    assert G.contains(start) and G.contains(end) and G.defineSortCriteria(option = criteria)
    pq = []
    out = []
    heapq.heappush(pq,(0,(start , [start])))
    reached = set()
    while len(pq) > 0:
        cost ,(cur,path) = heapq.heappop(pq)
        out.append(cur)
        if cur == end:
            if verbose: print(f'Out order: {" ".join(out)}') 
            return path
        if cur not in reached:
            reached.add(cur)
            for node in G.adj(cur):
                heapq.heappush(pq,(cost + G.cost(cur,node),(node , path + [node])))
    if verbose: print(f'\nOut order: {" ".join(out)}') 
    return None

In [16]:
path = UCSearch(G,'A','J',verbose = True)
if path: print('\nFinal path found: '+' -> '.join(path))

Out order: A E H B E G D H D F C C G A E I F G I J

Final path found: A -> B -> I -> J


## 4. Búsqueda $A^*$

Finalmente ejecute búsqueda $A∗$. Tenga en cuenta que si hay empates en la cola de prioridad se extrae el estado con menor etiqueta.